## Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from torchvision.datasets import ImageFolder
from PIL import Image, UnidentifiedImageError

class SafeImageFolder(ImageFolder): # to skip the broken or empty img file
    def __getitem__(self, index):
        while True:
            try:
                return super(SafeImageFolder, self).__getitem__(index)
            except (UnidentifiedImageError, OSError) as e:
                print(f"Skipping broken image at index {index}: {e}")
                index = (index + 1) % len(self.imgs)



### Check if GPU is working or CPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Transformation and Resize

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


### Loading dataset

In [4]:
full_dataset = SafeImageFolder(root='/kaggle/input/disaster-images-dataset/Comprehensive Disaster Dataset(CDD)', transform=transform)


train_size = int(0.8 * len(full_dataset)) # 80% into training 
val_size = len(full_dataset) - train_size # 20% nto validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Defining model

In [5]:
model = models.densenet121(weights=True) # weights=true means weights are pre-trained


num_features = model.classifier.in_features
num_classes = len(full_dataset.classes)

model.classifier = nn.Linear(num_features, num_classes)

model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 221MB/s]


### Defining loss function

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


### Training part

In [7]:
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Skipping broken image at index 2455: cannot identify image file <_io.BufferedReader name='/kaggle/input/disaster-images-dataset/Comprehensive Disaster Dataset(CDD)/Human_Damage/02_0069.png'>
Epoch 1/15, Loss: 0.4101
Skipping broken image at index 2455: cannot identify image file <_io.BufferedReader name='/kaggle/input/disaster-images-dataset/Comprehensive Disaster Dataset(CDD)/Human_Damage/02_0069.png'>
Epoch 2/15, Loss: 0.2853
Skipping broken image at index 2455: cannot identify image file <_io.BufferedReader name='/kaggle/input/disaster-images-dataset/Comprehensive Disaster Dataset(CDD)/Human_Damage/02_0069.png'>
Epoch 3/15, Loss: 0.2434
Skipping broken image at index 2455: cannot identify image file <_io.BufferedReader name='/kaggle/input/disaster-images-dataset/Comprehensive Disaster Dataset(CDD)/Human_Damage/02_0069.png'>
Epoch 4/15, Loss: 0.2116
Skipping broken image at index 2455: cannot identify image file <_io.BufferedReader name='/kaggle/input/disaster-images-dataset/Comprehe

### Evulation

In [8]:
model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

### Calculate Metrices

In [9]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
mcc = matthews_corrcoef(all_labels, all_preds)

### AUC

In [10]:
try:
    auc = roc_auc_score(all_labels, all_probs, multi_class="ovr", average="weighted")
except:
    auc = None

In [11]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc if auc is not None else 'Not available'}")
print(f"MCC: {mcc:.4f}")

Accuracy: 0.9388
Precision: 0.9407
Recall: 0.9388
F1 Score: 0.9385
AUC: 0.9968090414951729
MCC: 0.8804


### Saving model

In [12]:

torch.save(model.state_dict(), 'densenet121_disaster.pth')


In [13]:
import os

print(os.listdir('/kaggle/working'))


['densenet121_disaster.pth', '.virtual_documents']
